In [1]:
# TODO LIST FROM GANHACKS
# https://github.com/soumith/ganhacks
# ✓ Normalize the inputs
# ✓ A modified loss function
# ✓ Use a spherical Z
# ✓ BatchNorm
# Aviod sparse gradients
# ✓ Use soft and noisy labels
# ✓ DCGAN
# Use stability tricks from RL
# ✓ Use SGD for discriminator ADAM for generator
# ✓ Add noise to inputs
# Batch Discrimination (for diversity)
# ✓ Use dropouts in G in both train and test phase

In [2]:
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import torchvision.datasets as dset
from Folder import ImageFeatureFolder

nc = 3
nz = 512
lr     = 0.0002
beta1  = 0.0   
beta2  = 0.99     
imageSize = 128
batchSize = 64

outf = "./celeba_result/"
des_dir = "./celeba/"

dataset = dset.ImageFolder(root=des_dir,
                        transform=transforms.Compose([
                            transforms.CenterCrop(178),
                            transforms.Resize(imageSize),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                        ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [3]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils
from Models import Generator, Discriminator
from VGG import VGG16Feature

In [4]:
netG = Generator()
netD = Discriminator()
vggnet = VGG16Feature()

criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
feature = torch.FloatTensor(batchSize, 512, 7, 7)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label_real = torch.FloatTensor(batchSize)
label_real_smooth = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
vggnet.cuda()
vggnet.eval()
input, feature, noise = input.cuda(), feature.cuda(), noise.cuda()
label_real, label_real_smooth, label_fake = label_real.cuda(), label_real_smooth.cuda(), label_fake.cuda()
fixed_noise = fixed_noise.cuda()

label_real.resize_(batchSize, 1).fill_(1)
label_fake.resize_(batchSize, 1).fill_(0)
label_real_smooth.resize_(batchSize, 1).fill_(0.9)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
label_real_smooth = Variable(label_real_smooth)
print()

D:\Ipython\caffeinism\DCGAN\custom_layers.py:131: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if initializer == 'kaiming':    kaiming_normal(self.deconv.weight, a=calculate_gain('conv2d'))
D:\Ipython\caffeinism\DCGAN\custom_layers.py:114: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if initializer == 'kaiming':    kaiming_normal(self.conv.weight, a=calculate_gain('conv2d'))
D:\Ipython\caffeinism\DCGAN\custom_layers.py:147: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if initializer == 'kaiming':    kaiming_normal(self.linear.weight, a=calculate_gain('linear'))


In [5]:
# netD.load_state_dict(torch.load(outf + 'netD_epoch_016.pth'))
# netG.load_state_dict(torch.load(outf + 'netG_epoch_016.pth'))

In [6]:
fixed_noise = Variable(fixed_noise)

# setup optimizer
# optimizerD = optim.SGD(netD.parameters(), lr = lr, momentum=0.9)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
schedulerD = optim.lr_scheduler.MultiStepLR(optimizerD, milestones=[4, 7, 11, 17], gamma=0.87)
schedulerG = optim.lr_scheduler.MultiStepLR(optimizerG, milestones=[4, 6, 8, 10, 12, 14, 17], gamma=0.87)

In [ ]:
import numpy as np
_d_ = None
def add_noise(x, d_fake):
    global _d_
    if _d_ is not None:
        _d_ = _d_ * 0.9 + torch.mean(d_fake).data[0] * 0.1
        strength = 0.2 * max(0.00001, _d_ - 0.5)**2
        z = torch.FloatTensor(*x.size()).normal_(0, 1) * strength
#         z = np.random.randn(*x.size()).astype(np.float32) * strength
        zv = Variable(z).cuda()
        return x + zv
    else:
        _d_ = 0.0
        return x


In [ ]:
niter = 100
d_fake_save = None
for epoch in range(niter):
    schedulerD.step()
    schedulerG.step()
#     if epoch < 17:
#         continue
    for i, data in enumerate(dataloader, 0):
        # train D
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real = real_cpu.cuda()
        input.resize_as_(real).copy_(real)
        
        inputv = Variable(input)
        inputv = add_noise(inputv, d_fake_save)
        feature = vggnet(inputv)
        
        d_real = netD(inputv, feature)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev, feature)
        
        d_fake_save = d_fake = netD(fake.detach(), feature)
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real_smooth) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake, feature)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%500 == 0:
            fake = netG(fixed_noise, feature)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))
            
    torch.save(netG.state_dict(), '%s/netG_epoch_%03d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%03d.pth' % (outf, epoch))

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0/100][0/3165] Loss_D: 0.6214 Loss_G: 0.4480 D(x): 0.4255 D(G(z)): 0.4096


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


[0/100][500/3165] Loss_D: 0.2409 Loss_G: 0.6314 D(x): 0.6232 D(G(z)): 0.2330
[0/100][1000/3165] Loss_D: 0.2864 Loss_G: 0.7458 D(x): 0.7274 D(G(z)): 0.2962
[0/100][1500/3165] Loss_D: 0.2300 Loss_G: 0.7988 D(x): 0.7867 D(G(z)): 0.3453
[0/100][2000/3165] Loss_D: 0.2540 Loss_G: 0.5191 D(x): 0.5905 D(G(z)): 0.2537
[0/100][2500/3165] Loss_D: 0.2813 Loss_G: 0.6493 D(x): 0.6224 D(G(z)): 0.3480
[0/100][3000/3165] Loss_D: 0.2951 Loss_G: 0.5887 D(x): 0.6351 D(G(z)): 0.3677
[1/100][0/3165] Loss_D: 0.2727 Loss_G: 0.5521 D(x): 0.6273 D(G(z)): 0.3452
[1/100][500/3165] Loss_D: 0.2617 Loss_G: 0.7095 D(x): 0.6948 D(G(z)): 0.3095
[1/100][1000/3165] Loss_D: 0.1715 Loss_G: 0.6547 D(x): 0.7007 D(G(z)): 0.2589
[1/100][1500/3165] Loss_D: 0.1858 Loss_G: 0.6902 D(x): 0.7202 D(G(z)): 0.2819
[1/100][2000/3165] Loss_D: 0.2653 Loss_G: 0.4097 D(x): 0.5124 D(G(z)): 0.1533
[1/100][2500/3165] Loss_D: 0.2624 Loss_G: 0.6005 D(x): 0.6249 D(G(z)): 0.3233
[1/100][3000/3165] Loss_D: 0.2092 Loss_G: 0.5795 D(x): 0.6359 D(G(z))

[15/100][500/3165] Loss_D: 0.1139 Loss_G: 0.9307 D(x): 0.8535 D(G(z)): 0.2265
[15/100][1000/3165] Loss_D: 0.1186 Loss_G: 0.9080 D(x): 0.8197 D(G(z)): 0.2376
[15/100][1500/3165] Loss_D: 0.1264 Loss_G: 0.6583 D(x): 0.6952 D(G(z)): 0.1020
[15/100][2000/3165] Loss_D: 0.2381 Loss_G: 1.2053 D(x): 0.9774 D(G(z)): 0.3892
[15/100][2500/3165] Loss_D: 0.1425 Loss_G: 0.9535 D(x): 0.8735 D(G(z)): 0.2624
[15/100][3000/3165] Loss_D: 0.1543 Loss_G: 0.9965 D(x): 0.8835 D(G(z)): 0.2793
[16/100][0/3165] Loss_D: 0.1271 Loss_G: 0.6355 D(x): 0.6587 D(G(z)): 0.0369
[16/100][500/3165] Loss_D: 0.1291 Loss_G: 0.6663 D(x): 0.6925 D(G(z)): -0.0029
[16/100][1000/3165] Loss_D: 0.1282 Loss_G: 0.7312 D(x): 0.7221 D(G(z)): 0.0640
[16/100][1500/3165] Loss_D: 0.1100 Loss_G: 0.7635 D(x): 0.7397 D(G(z)): 0.1204
[16/100][2000/3165] Loss_D: 0.0952 Loss_G: 0.9073 D(x): 0.8360 D(G(z)): 0.1705
[16/100][2500/3165] Loss_D: 0.1241 Loss_G: 0.7154 D(x): 0.7149 D(G(z)): 0.1128
[16/100][3000/3165] Loss_D: 0.1063 Loss_G: 0.8659 D(x): 

[30/100][0/3165] Loss_D: 0.0918 Loss_G: 0.7599 D(x): 0.7571 D(G(z)): 0.0973
[30/100][500/3165] Loss_D: 0.0897 Loss_G: 0.6813 D(x): 0.7023 D(G(z)): 0.0340
[30/100][1000/3165] Loss_D: 0.0843 Loss_G: 0.8088 D(x): 0.7823 D(G(z)): 0.0836
[30/100][1500/3165] Loss_D: 0.0889 Loss_G: 0.8853 D(x): 0.8278 D(G(z)): 0.1365
[30/100][2000/3165] Loss_D: 0.0696 Loss_G: 0.8821 D(x): 0.8200 D(G(z)): 0.0610
[30/100][2500/3165] Loss_D: 0.1017 Loss_G: 1.2029 D(x): 0.9132 D(G(z)): 0.1338
[30/100][3000/3165] Loss_D: 0.0745 Loss_G: 1.1442 D(x): 0.9218 D(G(z)): 0.1292
[31/100][0/3165] Loss_D: 0.0636 Loss_G: 0.8458 D(x): 0.7979 D(G(z)): -0.0429
[31/100][500/3165] Loss_D: 0.0712 Loss_G: 0.8968 D(x): 0.8307 D(G(z)): 0.1150
[31/100][1000/3165] Loss_D: 0.1006 Loss_G: 1.0465 D(x): 0.8866 D(G(z)): 0.1738
[31/100][1500/3165] Loss_D: 0.0846 Loss_G: 0.7162 D(x): 0.7280 D(G(z)): 0.1020
[31/100][2000/3165] Loss_D: 0.0928 Loss_G: 0.9708 D(x): 0.8604 D(G(z)): 0.2150
[31/100][2500/3165] Loss_D: 0.1042 Loss_G: 0.6721 D(x): 0.7

[44/100][3000/3165] Loss_D: 0.0650 Loss_G: 0.9215 D(x): 0.8489 D(G(z)): 0.0686
[45/100][0/3165] Loss_D: 0.0603 Loss_G: 1.0598 D(x): 0.9171 D(G(z)): 0.1427
[45/100][500/3165] Loss_D: 0.0800 Loss_G: 0.9534 D(x): 0.8319 D(G(z)): 0.1238
[45/100][1000/3165] Loss_D: 0.0347 Loss_G: 0.8972 D(x): 0.8337 D(G(z)): 0.0168
[45/100][1500/3165] Loss_D: 0.0659 Loss_G: 0.8714 D(x): 0.8123 D(G(z)): 0.0435
[45/100][2000/3165] Loss_D: 0.1407 Loss_G: 1.3108 D(x): 0.9780 D(G(z)): 0.2760
[45/100][2500/3165] Loss_D: 0.0728 Loss_G: 0.8498 D(x): 0.8075 D(G(z)): 0.0663
